Many examples are extensions of https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash
which are from [Interactive Dashboards and Data Apps with Plotly and Dash](https://www.packtpub.com/product/interactive-dashboards-and-data-apps-with-plotly-and-dash/9781800568914?utm_source=github&utm_medium=repository&utm_campaign=9781800568914)

In [25]:
import intersystems_iris.dbapi._DBAPI as dbapi
from dotenv import load_dotenv
load_dotenv()
import os

# See example from https://github.com/intersystems-community/intersystems-irispython
# Is some issues with the other driver, so use this one.

password = os.getenv("SQL_PASSWORD")
user = os.getenv("SQL_USERNAME")
host = os.getenv("SQL_SERVER")
namespace = os.getenv("SQL_NAMESPACE")
port = os.getenv("SQL_PORT")
if isinstance(port, str):
    port = int(port)

config = {
    "hostname": host,
    "port": port,
    "namespace": namespace,
    "username": user,
    "password": password,
}

try:
    conn = dbapi.connect(**config)
    print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

# create a cursor
cursor = conn.cursor()
cursor


Connection successful!


In [26]:
import pandas as pd

sql = """
      select ResourceType, count(*) from HSFHIR_X0001_R.Rsrc group by ResourceType
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ResourceType,Aggregate_2
0,Binary,100
1,DiagnosticReport,8
2,DocumentReference,8
3,Encounter,12
4,Observation,2
5,Organization,15
6,Patient,10
7,ServiceRequest,11
8,Specimen,3
